In [1]:
import (
	"encoding/json"
    . "github.com/FactomProject/ptnet-eventstore/contract"
	"github.com/FactomProject/ptnet-eventstore/finite"
	. "github.com/FactomProject/ptnet-eventstore/identity"
	. "github.com/FactomProject/ptnet-eventstore/ptnet"
	"github.com/FactomProject/ptnet-eventstore/x"
)

In [2]:
// the state machine definition composes the rules of the contract
// machine must be designed to always end in a halting state
// this protocol requires the machine to be halted before tokens are released
//
// NOTE:
// this type of state machine classified as a https://en.wikipedia.org/wiki/Vector_addition_system
// and can also be represented graphically as a https://en.wikipedia.org/wiki/Petri_net
m := Machine{
    // define the inital state - a nice convention follow
    // is to set each variable to the max possible value
    // that can be achieved during valid contract execution
    // in this contract no value ever exceeds 1
	//                   00 01 02 10 11 12 20 21 22 O  X $O $X $DEP
	Initial: StateVector{1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1},

	Transitions: map[string]Transition{
        // each transition defines its effect 
        // on every offset 'variable' declared in the state vector
        //
        // NOTE: this is mathematical vector addition at work
        // each vector is 'executed' by adding it to the preceding state
		//                00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
        // by convention a contract must have a BEGIN/"EXEC" transaction
        // this action is executed when the contract is first offered
		BEGIN: Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, -1},
        // moves for player X
		"X00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, 1, -1, 0, 0, 0},
		"X20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, 1, -1, 0, 0, 0},
		"X21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0, 0},
		"X22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, 1, -1, 0, 0, 0},
        // moves for player O
		"O00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 1, 0, 0, 0},
		"O20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 1, 0, 0, 0},
		"O21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 1, 0, 0, 0},
		"O22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1, 0, 0, 0},

        // moves that halt game and unlock tokens for winner
        // or refund to the depositor
		"WINX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0},
		"WINO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0},
		"ENDX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1},
		"ENDO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1},
	},
}

In [3]:
// contract declaration adds more rules in addition to the state machine
gameContract := Declaration{
    // inputs are similar to most cryptocurrency rules
    // and represent a type of 'pay-to-script' functionality
    // tokens are lock away in the contract until the blockheight is exceeded (invalid/fault)
    // or the contract halts (valid completion)
	Inputs: []AddressAmountMap{
		AddressAmountMap{Address[DEPOSITOR], 1}, // deposit 1 token
	},
    // outputs are referenced by Guards and Conditions
    // and are used as an Access Control List (ACL)
    // to define roles of users that are allowed to interact
    // with the contract in addition to defining which users
    // can withdraw tokens from a completed/halted contract
	Outputs: []AddressAmountMap{ // array of possible redeemers
		AddressAmountMap{Address[DEPOSITOR], 1},
		AddressAmountMap{Address[PLAYERX], 1},
		AddressAmountMap{Address[PLAYERO], 1},
	},
	BlockHeight: 60221409,      // deadline to achieve halting state
	Salt:        "|RANDOM|",    // random salt
	ContractID:  "|OctoeContractID|",// unique ID for this contract instance
	Schema:      "octoe-v1",    // versioned contract schema
	State:       m.Initial,     // state machine initial state
	Actions:     m.Transitions, // state machine defined transitions
	Guards: []Condition{ // guards restrict state machine actions
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, // Admin/Depositor
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0},// PlayerX turn
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0},// PlayerO turn
	},
	Conditions: []Condition{ // redeem conditions to unlock input tokens
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1},// no win/refund deposit
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0},// X win
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0},// O win 
	},
}

In [4]:
// use declaration to construct an offer
// this is a data wrapper that is used to isolate transaction ops from declaration ops
// NOTE: mostly this is just an artifact of this implementation not especially meaningful
offer := finite.Offer{
    ChainID: CHAIN_ID,
	Declaration: gameContract,
}

In [5]:
offer.String()


ChainID: |ChainID|
Inputs: 
	Address: dd808299f72f362e3cab3b8d1955505133ed1d1c72db295108f8896f767aeb7b Amount: 1 
Outputs: 
	Address: dd808299f72f362e3cab3b8d1955505133ed1d1c72db295108f8896f767aeb7b Amount: 1 
	Address: 947b35b8e4dad5957e24ed0bf02903c6390863753c753106a778111000cefa04 Amount: 1 
	Address: 1c6334d78fd8a6879c7143d2cf16ad7559d5e37c1587c2025fa44b41be597db7 Amount: 1 
BlockHeight: 60221409
Salt: |RANDOM|
ContractID: |OctoeContractID|
Schema: octoe-v1
State: [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actions: 
	ENDO: [0 0 0 0 0 0 0 0 0 -1 0 0 0 1]
	ENDX: [0 0 0 0 0 0 0 0 0 0 -1 0 0 1]
	EXEC: [0 0 0 0 0 0 0 0 0 -1 0 -1 -1 -1]
	O00: [-1 0 0 0 0 0 0 0 0 -1 1 0 0 0]
	O01: [0 -1 0 0 0 0 0 0 0 -1 1 0 0 0]
	O02: [0 0 -1 0 0 0 0 0 0 -1 1 0 0 0]
	O10: [0 0 0 -1 0 0 0 0 0 -1 1 0 0 0]
	O11: [0 0 0 0 -1 0 0 0 0 -1 1 0 0 0]
	O12: [0 0 0 0 0 -1 0 0 0 -1 1 0 0 0]
	O20: [0 0 0 0 0 0 -1 0 0 -1 1 0 0 0]
	O21: [0 0 0 0 0 0 0 -1 0 -1 1 0 0 0]
	O22: [0 0 0 0 0 0 0 0 -1 -1 1 0 0 0]
	WINO: [0 0 0 0 0 0 0 0 0 0

In [6]:
// check to see if contract has been recorded yet (it hasn't)
Exists(gameContract.Schema, gameContract.ContractID)

false

In [7]:
// to begin contract execution an - offer transaction is made
// by convention the initial state should be set to the upper bound
// for all the variables used by the contract
// in this contract no variable is ever > 1
//
// NOTE by design - the state machine reports any output values < 0 as invalid
// see: https://en.wikipedia.org/wiki/Vector_addition_system for more details
txn := finite.OfferTransaction(offer, Private[DEPOSITOR])
txn.String() // notice the event below contains the json-encoded contract declaration


Timestamp:   1542725960248198058
Schema:      octoe-v1
Action:      EXEC
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
OutputState: [1 1 1 1 1 1 1 1 1 0 1 0 0 0]
Payload:
	7b0a2020202022696e70757473223a205b0a20202020202020207b0a2020202020202020202020202261646472657373223a2022335943436d6663764e693438717a754e47565651555450744852787932796c524350694a62335a363633733d222c0a20202020202020202020202022616d6f756e74223a20310a20202020202020207d0a202020205d2c0a20202020226f757470757473223a205b0a20202020202020207b0a2020202020202020202020202261646472657373223a2022335943436d6663764e693438717a754e47565651555450744852787932796c524350694a62335a363633733d222c0a20202020202020202020202022616d6f756e74223a20310a20202020202020207d2c0a20202020202020207b0a2020202020202020202020202261646472657373223a20226c487331754f5461315a562b4a4f304c38436b44786a6b495933553864544547703367524541444f2b67513d222c0a20202020202020202020202022616d6f756e74223a20310a20202020202020207d2c0a202020

In [8]:
// check to see if our contract has been recorded
Exists(gameContract.Schema, gameContract.ContractID)

true

In [9]:
// show that contract is still open
IsHalted(gameContract)

false

In [10]:
// show that tokens are locked
// even original depositor cannot withdraw tokens
CanRedeem(gameContract, Public[DEPOSITOR])

false

In [11]:
// Helper function to make it easier to compose transactions in this example
func commit(action string, key PrivateKey, payload []byte) (finite.Transaction, error) {
	pub := PublicKey{}
	copy(pub[:], x.PrivateKeyToPub(key[:])) // get public key
	return finite.ExecuteTransaction(finite.Execution{
		Command: Command{
			ChainID:    CHAIN_ID,// which chain to add entry
			ContractID: "|OctoeContractID|",
			Schema:     OctoeV1, // state machine version
			Action:     action,  // state machine action
			Amount:     1,       // triggers input action 'n' times amount
			Payload:    payload, // arbitrary data optionally included
			Pubkey:     pub,     // must match key in contract declaration
		},
	}, key)
}

In [12]:
// an event with an arbitrary payload attached
// on-chain it is expected that this payload is used to attach
// additional supporting data or evidence that 
// an action is being executed according to the intent of the contract
payload, _ := json.Marshal([]string{"hello", "world"})
event1, _ := commit("X11", Private[PLAYERX], payload)
event1.String()


Timestamp:   1542725960252192977
Schema:      octoe-v1
Action:      X11
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 1 1 1 1 1 1 1 1 0 1 0 0 0]
OutputState: [1 1 1 1 0 1 1 1 1 1 0 0 0 0]
Payload:
	5b2268656c6c6f222c22776f726c64225d
digest:
	d1b90bd64f9f5263ca152b8d39207faad90b9c577dcc70da6e61788bb528c9a4
pubkeys: 
	f09b40a1ceb3cd37f7a3584bbcccde13ad1d8313254f8d6fb3870786264179bf
signatures: 
	310801a6686a17a1ef5cf7216e0084bbab5a1d28355088466a904aa6b0012905301426c637513bb18aea59c2b252178ea8f1b3f5c49fa81b73a8af73e9f0bb0f


In [13]:
// show an invalid move
// according to this state machine players must take turns
_, err := commit("X00", Private[PLAYERX], nil)
err

invalid output: -1 offset: 10

In [14]:
// another invalid move - here we try to sign with an incorrect key
// contract makes players sign each event
_, err := commit("O01", Private[PLAYERX], nil)
err

failed guard condition

In [15]:
// valid
event2, _ := commit("O01", Private[PLAYERO], nil)
event2.String()


Timestamp:   1542725960259065819
Schema:      octoe-v1
Action:      O01
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 1 1 1 0 1 1 1 1 1 0 0 0 0]
OutputState: [1 0 1 1 0 1 1 1 1 0 1 0 0 0]
Payload:
	
digest:
	1ba445168c2bdfba147bb1b7241d7f36011a5ef897fad331d40914b6b70a59ad
pubkeys: 
	843ec312a748b8c42989281ca090d0c659f6a03e8346edf7d4d512a8b76a76ef
signatures: 
	bff9cd82324458f2270dea8ba977b55e3bdf53088fccaa915233c0323ff34bc2286abff76134cbea054722606f4ceca445eb7ebfa96a5956063593bc48f0150f


In [16]:
// invalid - state machine ensures move is used only once
// move X11 is already taken as seen above
_, err := commit("X11", Private[PLAYERX], nil)
err

invalid output: -1 offset: 4

In [17]:
// valid
event3, _ := commit("X00", Private[PLAYERX], nil)

[]interface{}{event6.Action, event6.InputState, "=>", event6.OutputState}

ERROR: repl.go:4:15: undefined identifier: event6

In [18]:
// valid
event4, _ := commit("O02", Private[PLAYERO], nil)
event4.String()


Timestamp:   1542725960261051859
Schema:      octoe-v1
Action:      O02
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 0 1 1 0 1 1 1 1 0 1 0 0 0]
OutputState: []
Payload:
	
digest:
	
pubkeys: 
signatures: 


In [19]:
// valid
event5, _ := commit("X22", Private[PLAYERX], nil)
event5.String()


Timestamp:   1542725960261727195
Schema:      octoe-v1
Action:      X22
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 0 1 1 0 1 1 1 1 0 1 0 0 0]
OutputState: [1 0 1 1 0 1 1 1 0 1 0 0 0 0]
Payload:
	
digest:
	0cb4783005e6205c9f192114efee3dc95b99852d8e4695867c96ee1891380006
pubkeys: 
	f09b40a1ceb3cd37f7a3584bbcccde13ad1d8313254f8d6fb3870786264179bf
signatures: 
	8e91af977f68b8e87cd34f954dc638f1358959e082608b35e74f418441b5f8cab0a3d46256aee24d4f24501dbc3fca743990ec91e51d9dc8e69dcb51caa4ac08


In [20]:
// game is over
// contract depositor confirms the winner
// this halts the state machine thereby unlocking the deposited tokens
event6, _ := commit("WINX", Private[DEPOSITOR], nil)
event6.String()


Timestamp:   1542725960262806767
Schema:      octoe-v1
Action:      WINX
Oid:         |OctoeContractID|
Value:       1
InputState:  [1 0 1 1 0 1 1 1 0 1 0 0 0 0]
OutputState: [1 0 1 1 0 1 1 1 0 0 0 0 1 0]
Payload:
	
digest:
	8b8d594bb6d20d4979860451573a929e29e39d764ba3732d1d4f0f444c8d06f0
pubkeys: 
	824e2a348148276ccc0013c12bd43eb07b4f486decef781cbff6ecb72105cdfc
signatures: 
	ad4b5121bfbfa931fdefa93671c5ec22337c389a34bca93e3fdb493af59dec4258e4b1281c0e9ff1a8cada9a49222dd0356ed13b216ab16bd65a094b9ddba00c


In [21]:
// contract is only redeemable after halting
IsHalted(gameContract)

true

In [22]:
// only the winner can redeem the deposit
CanRedeem(gameContract, Public[PLAYERX])

true

In [23]:
// loser has no claim
CanRedeem(gameContract, Public[PLAYERO])

false